In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import re
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'Times New Roman'

In [3]:
# !pip install gensim библиотека для «Тематического моделирования»
# !pip install razdel- токенизация
# !pip install pymorphy2 - приведение слов в нормальную форму
# !pip install nltk - токенизация лемматизация и т.д.

In [4]:
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
from gensim.models import LdaModel
from gensim.test.utils import datapath

In [5]:
# новости
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [6]:
# Загрузим пользователей и списки последних прочитанных новостей
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [7]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))
morph = pymorphy2.MorphAnalyzer()

151


In [8]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [9]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [10]:
%%time
#Запускаем лемматизацию текста.
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 22s, sys: 239 ms, total: 4min 22s
Wall time: 4min 23s


In [11]:
#сформируем список текстов
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [12]:
%%time
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 32.6 s, sys: 24 ms, total: 32.6 s
Wall time: 32.6 s


In [13]:
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [14]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(3, 0.1608058),
 (8, 0.19282326),
 (11, 0.07835247),
 (20, 0.24427506),
 (22, 0.3025198)]

In [15]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: риск километр болезнь объяснять сведение сша белорусский
topic_1: столетие болгария аргентина прибегать 2030 nnnn сестра
topic_2: век офицер мальчик нанести мышь свидетель фаза
topic_3: золото комплексный нил красноярский статистика 1,9 победитель
topic_4: население конкурс кровь запустить супруг фотография эксперимент
topic_5: земля мозг станция вода новый пациент км
topic_6: газ европа превысить северный армения эстония литва
topic_7: погибнуть произойти тело экипаж пострадать жертва способность
topic_8: цена ракета рост рынок эксперт млн технология
topic_9: украина украинский задержать мужчина киев миллиард уголовный
topic_10: военный помощь район область граница 2017 операция
topic_11: nn место тыс рейтинг первый южный сократиться
topic_12: 2019 строительство улица столица увеличение объект ступень
topic_13: ребёнок россия гражданин статья газета nn писать
topic_14: исследование экономика возраст университет анализ родитель планета
topic_15: новый тыс большой первый автор 

In [16]:
# функция, которая будет возвращать векторное представление новости
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [17]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0781,0.0661,0.0000,0.0000,0.0282,0.0000,0.0000,0.8193,0.0000
1,4896,0.0000,0.0000,0.0000,0.2502,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.7258,0.0000,0.0000,0.0000,0.0000
2,4897,0.0000,0.0000,0.0000,0.1608,0.0000,0.0000,0.0000,0.0000,0.1928,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.2443,0.0000,0.3025,0.0000,0.0000
3,4898,0.0000,0.0000,0.0000,0.0562,0.0000,0.0186,0.0000,0.0000,0.0000,...,0.1771,0.0349,0.0000,0.0000,0.0000,0.6392,0.0000,0.0641,0.0000,0.0000
4,4899,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.3904,0.0000,0.1386,0.4449,0.0000


In [18]:
# векторные представления пользователей
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.34110534,
       0.        , 0.03112972, 0.        , 0.        , 0.        ,
       0.        , 0.12814406, 0.06438516, 0.        , 0.        ,
       0.23586519, 0.        , 0.        , 0.06801692, 0.        ,
       0.        , 0.11934277, 0.        , 0.        , 0.        ])

******

1. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
Повторить п.2, но используя уже не медиану, а max

2. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [20]:
def get_user_embedding(user_articles_list, agg_type='mean'):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if agg_type == 'mean':
        user_vector = np.mean(user_vector, 0)
    if agg_type == 'median':
        user_vector = np.median(user_vector, 0)
    if agg_type == 'max':
        user_vector = np.max(user_vector, 0)
    return user_vector

In [35]:
agg_list = ['mean', 'median', 'max']
result_dict = {}
result_dict['metrics'] = ['thresholds', 'fscore', 'precision', 'recall', 'roc_auc_score']
for agg_type in agg_list:
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(
        lambda x: get_user_embedding(x, agg_type=agg_type), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] +
                                      ['topic_{}'.format(i) for i in range(25)]]
    
    target = pd.read_csv("users_churn.csv")
    X = pd.merge(user_embeddings, target, 'left')
    
    # разделение на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)
    # обучение
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    # предсказания
    y_pred = logreg.predict(X_test)
    y_proba = logreg.predict_proba(X_test)[:, 1]
    # метрики качества
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    result_dict[agg_type] = [thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score(y_test, y_proba)]
df_result = pd.DataFrame(result_dict)
df_result

,metrics,mean,median,max
0,thresholds,0.2277,0.2658,0.3801
1,fscore,0.6380,0.7117,0.8008
2,precision,0.5361,0.6436,0.8058
3,recall,0.7878,0.7959,0.7959
4,roc_auc_score,0.9338,0.9626,0.9771


Отток в большинстве случаев определяется одной темой, важной для пользователя. При использовании mean в качестве усреднения занижается важность этой темы для (разносторонних) пользователей, которые читают новости и на другие тематики тоже. Использование max в качестве усреднения более полно характеризует интресы пользователя. Медиана - промежуточный вариант.